In [ ]:
import sys
sys.path.insert(1, '../')

from helper_functions import ppc
from models import models, guides
from plots import plots 
import predictors
from models import posterior
import pyro.contrib.autoguide as autoguide

import torch 
import numpy as np
from plotly.offline import init_notebook_mode

import data

init_notebook_mode(connected=True)

# Car Crash Prediction in Manhattan using Variational Inference

Over 1.73 million crash incidents have been reported in NYC since 2012. The magnitude of this number indicates the importance of developing and understanding of the patterns that drive this phenomenon. With the aim of achieving this objective we developed various probability models that describe how the phenomenon occurs on a day to day level in different regions within Manhattan. We perform inference on our model using Stochastic Variational inference.
## Data description
We used four types of data. 
1. Location and time of all car crashes reported in Manhattan from [NYC OpenData](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95)

2. Daily average temperature, wind speed, rain volume and snow depth at JFK airport collected from [NCDC](https://www.ncdc.noaa.gov/cdo-web/search)

3. Intersection location and characteristics for all intersection of manhattan from [Kaggle](https://www.kaggle.com/crailtap/street-network-of-new-york-in-graphml)

4. Annual average daily traffic for all road segments in manhattan for which this information was available, taken from [NY government](https://data.ny.gov/Transportation/Annual-Average-Daily-Traffic-AADT-Beginning-1977/6amx-2pbv) 


Below we plot some graphs that show different aspects of our data. 

In [ ]:
accidents, preds  = data.get_data()
plots.make_heat_map()

In [ ]:
plots.make_mean_log_mean(accidents)

The plot below plots accidents through time and uses a smoother to detect trends through time.

In [ ]:
plots.make_time_series(accidents)

## Data aggregation
To aggregate the data we decided to map each accident to the nearest intersection. We decided to use the average AADT over the period from 2014 to 2019 for each road as there was missing data. We matched the AADT of the roads to the intersections by proximity. Roads without AADT data available were remove. For interpretability and inference purposes we used normalized $\log$(AADT).

## Model definition
The main model that we used is a poisson log-normal model. Let $m$ denote the number of days, $n$ the number of sites or regions, and $k$ the number of predictors. Then we let  $Y_{ij} $ denote the number of car crashes in region $i$ on day $j$.  Call $\beta \in \mathbb{R}^k$ the regression coefficients and $X_{ij} \in \mathbb{R}^k$ the predictors for site $i$ on day $k$. We then assume that the data is generated from the following model 


$$\epsilon \sim \mathcal{N}(0, 10^2 * I_n)\\
\beta \sim \mathcal{N}(0, 5^2 * I_k)\\
\log(\theta_{ij}) = X_{ij}^\top \beta + \epsilon_{i}\\
Y_{ij} \sim \text{Poisson}(\theta_{ij})$$

## Preliminary Investigations: Inference Method and Model Choices

Before working on the real data, we implemented our main models and fit them to synthetically generated data from the model itself. We found that the models were able to recover the correct beta’s even when the generated data had similar sparsity as the real data. We learned two things from this:

- Initially, we were planning on fitting a conditional autoregressive model using spatial correlation. However, we found this to be prohibitively computationally expensive regardless of inference method due to computations involving large spatial matrices. Due to the large number of intersections, this was not a feasible model. As such, we decided to use the poisson-lognormal model.

- Initially, we wanted to use MCMC as our inference method but we soon found this to be computationally infeasible due to the large amount of data. Instead, we choice to use stochastic variational inference using Pyro's automatic guide generation. 

- We note hencefoth that every inference was performed using a meanfield variational family of the same family of the prior. 

## Base model 
For our base model we assume that $k = 1$. This is equivalent to assuming no structure across nodes and simply modelling every region independently and an intercept. 


In [ ]:
pred_names = []
kappa = 0.55
t_0 = 20
loss, guide = models.train_log_linear_random_init(accidents,
                        preds,
                        pred_names, 
                        kappa=kappa,
                        t_0=t_0, 
                        max_iters=1000)
plots.plot_svi_loss(loss)


In [ ]:
selection = predictors.get_some_predictors(preds, pred_names)
predict = posterior.Predict(models.log_linear_model, guide, 300)
samples = predict(accidents.shape[0], accidents.shape[1], selection.shape[2], torch.Tensor(selection))

### PPC 

We now check whether our model is able to generate similar data to the actual data. To do so, we conduct two posterior predictive checks:

- The actual number of accidents at an intersection and the empirical distribution constructed from the posterior predictictive samples. We expect a good model to have the mean of the empirical distribution to coincide with the actual data.
- The max number of accidents during one day at a given intersection during some period of time at the emprical distribution of this same quantity. 

In [ ]:
ppc.plot_total_distributions(samples['accidents'].detach().numpy(), accidents, (2,2), 6)

In [ ]:
ppc.plot_max(samples['accidents'].detach().numpy(), accidents)

# Explaining the disparity with maximum
As the plots above show, the maximum number of crashes at a particular site is not captured well. Thus, we decided to fit negative bionomial model as it allows for an excess kurtosis greater than that of a poisson. 
We modeled the mean with the same structure as our main model and used a uniform prior over the parameter $p$ (success probability in each trial). In particular $\log(\theta)$ is defined as before and $\theta$ denotes mean of binomial but now we let $p_i \sim \text{Beta}(\alpha_0, \alpha_1)$. Both of these values parametrize the distribution. We choose here $\alpha_0 = \alpha_1 = 0$ and we assume that $Y_{ij}  \sim \text{Binom}(\theta_{ij}, p_i).$ under this particular parameterization of the binomial distribution.

In [ ]:
num_days = 365
selection = predictors.get_some_predictors(preds, [])[:,:num_days]
kappa = 0.5 + 1e-10
t_0 = 40
model_args = {'num_sites':accidents.shape[0],
             'num_days':num_days,
              'num_predictors':selection.shape[-1], 
              'predictors': torch.Tensor(selection),
             'data': torch.Tensor(accidents[:,:num_days])}

losses = models.train(models.negative_binomial_log_linear_model,
                      guides.negative_binomial_guide,
                      kappa = kappa,
                      t_0 = t_0,
                      model_args = model_args,
                      max_iters = 100
                     )
plots.plot_svi_loss(losses)

In [ ]:
predict = posterior.Predict(models.negative_binomial_log_linear_model, guides.negative_binomial_guide, 400)
binsamples = predict(accidents.shape[0], num_days, selection.shape[2], torch.Tensor(selection))

In [ ]:
ppc.plot_total_distributions(binsamples['accidents'].detach().numpy(),accidents[:,:num_days],(2,2),12)


In [ ]:
ppc.plot_max(binsamples['accidents'].detach().numpy() ,accidents)

To verify whether these results made sense we decided to compare the variance to the mean across sites. We determined that they were a poor approximation to reality. Moreover, we found three outliers which accounter for days with more than six crashes. We decide against using a negative binomial model and choose poisson as it approximates data more accurately. 

In [ ]:
plots.plot_mean_variance_line(binsamples['accidents'].detach().numpy()[np.random.randint(0,400)])

In [ ]:
plots.plot_mean_variance_line(accidents)

## Intersection Model 
The next step is to add local data to the nodes like AADT and whether the node is an intersection of 3 or more roads (as sometimes the node is simply a corner). After fitting the model, we see that the PPC sample has a similar empirical distribution of the mean difference as the actual data. Thus, this model is better able to capture node differeneces. However, we find it surprising that the AADT has a negative coefficient because intuitively we expected higher traffic to translate to greater accidents. Perhaps, this is due to higher traffic intersections having certain characteristics that make accidents less likely or AADT (as measured by us) capturing additional information. 

In [ ]:
pred_names = ['aadt','is_intersection']
kappa = 0.5
t_0 = 20
loss, guide = models.train_log_linear_random_init(accidents,
                        preds,
                        pred_names, 
                        kappa=kappa,
                        t_0=t_0, 
                        max_iters=3000)
plots.plot_svi_loss(loss)

In [ ]:
selection = predictors.get_some_predictors(preds, pred_names)
predict = posterior.Predict(models.log_linear_model, guide, 400)
insamples = predict(accidents.shape[0], accidents.shape[1], selection.shape[2], torch.Tensor(selection))

In [ ]:
plots.plot_betas(insamples['betas'].detach().numpy(), pred_names)

## Additional PPC
We introduce two additional PPCs to our model: 
- We plot the quantiles from posterior predictive of the a svagol filtered plot of total car crashes in Manhattan per day against the same quantity observed in data. We use window of 61 and polynomial of 3. 

- We introduce a further check to test whether a given categorical daily predictor affects accident rates. Assume a histogram of the difference between the mean accident for each node across days when the categorical predictor is 1 and the corresponding mean when it is 0. If said predictor does indeed affect the accident rate, we expect the histogram to have non-zero mean and be skewed . We can conduct  a check  by plotting skewness  of the histogram described above calculated from posterior samples according to truth. This is done below. This strategy works generally. 


In [ ]:
ppc.plot_time_trend(insamples['accidents'].detach().numpy(),accidents)

In [ ]:
ppc.plot_mean_difference(accidents,preds[:,0,2],0, title='Mean difference of is_intersection in real dataset')


In [ ]:
ppc.plot_skew(insamples['accidents'], accidents,preds[:,0,2],0 ,title='Skewness of intersection mean differences')

We see that the skew plot is adequate but time not very much. 

## Weather-Intersection model 

This prompts us to add predictors related to the day. We add temperature, snow depth, wind and precipitation as predictors as these are natural factors that one would expect to affect accident rates. After fitting the model using SVI, we apply the above PPC again. 



We find that the second criteria is better captured by this model but not perfectly. Due to the non-stationary confidence interval implied by our modified model, it is better able to capture the time-trends in the aggregate data. 


In [ ]:
pred_names = ['aadt','is_intersection', 'wind','snow_depth', 'temperature','precipitation']
kappa = 0.55
t_0 = 30
loss, guide = models.train_log_linear_random_init(accidents,preds,pred_names,kappa=kappa,t_0=t_0,max_iters=2000)
plots.plot_svi_loss(loss)

In [ ]:
selection = predictors.get_some_predictors(preds, pred_names)
predict = posterior.Predict(models.log_linear_model, guide, 400)
wsamples = predict(accidents.shape[0], accidents.shape[1], selection.shape[2], torch.Tensor(selection))

In [ ]:
ppc.plot_time_trend(wsamples['accidents'].detach().numpy(), accidents)

In [ ]:
ppc.plot_skew(wsamples['accidents'], accidents,preds[0,:,5] > 0,1 ,title='Skewness of mean differences Snow')

The previous checked all yielded adequate results. Our betas obtained are the following

In [ ]:
plots.plot_betas(wsamples['betas'].detach().numpy(), pred_names)

In [ ]:
We see that rain has a significant positive coefficient indicating that rain increases ratee

## Complication with SVI

When fitting the above model we noticed that running inference multiple times led to inconsistent results from the PPC. Sometimes, the trend lines would fit well and other times it would not. After studying the corresponding ELBO curves, we concluded that this was due to the ELBO not converging even after a significant number of iterations (~5000 iterations). We found that this was due to bad initialization of the guide. To prevent this, we initialize the guide a hundred times and select the initialization based on the lowest ELBO. This led us to consistent results from the PPC. The PPCs were crucial for us to understand this result and although not explicitely mentioend here they were used to check inference and verify model convergence.  


## Future Directions and Other attempts
We noticed two things that prompted us to define a mixture model:
We observed that for certain predictors, mean-field SVI parameters tended to converge to two or three different peaks. This may indicate that the posterior is multimodal and may reduce to unimodal when considering a single cluster assignment
For certain predictors, the PPC for mean difference seems to have smaller peaks
In the mixture model, we assign the nodes to several different sets of betas and the rest is identical to our main model. We define this model in models/models.py and we fit this using MAP showing heterogeneous results. However, we did not have enough time to properly fit and criticize the results of this model. So, we believe this to be an extremely promising future direction that we plan to look into.